In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import joblib

## Loading the features 

In [2]:
df_xgb = pd.read_csv("../data/features/features_XGBOOST.csv")
df_mlp = pd.read_csv("../data/features/features_MLP.csv")
df_lstm = pd.read_csv("../data/features/features_LSTM.csv")

print("Datasets loaded successfully.")

Datasets loaded successfully.


## Preparing Model for training

In [3]:
def ts_split(df, target="target", split=0.8):
    split_idx = int(len(df) * split)
    train = df.iloc[:split_idx]
    test  = df.iloc[split_idx:]

    X_train = train.drop(columns=[target])
    y_train = train[target]

    X_test = test.drop(columns=[target])
    y_test = test[target]

    return X_train, X_test, y_train, y_test

## Training XGBoost Regressor

In [4]:
df_xgb = df_xgb.drop(columns=["timestamp"], errors="ignore")

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = ts_split(df_xgb)

model_xgb = xgb.XGBRegressor(
    n_estimators=350,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror"
)

In [5]:
model_xgb.fit(X_train_xgb, y_train_xgb)

pred_xgb = model_xgb.predict(X_test_xgb)

In [6]:
# --- METRICS ---
rmse_xgb = np.sqrt(mean_squared_error(y_test_xgb, pred_xgb))
mae_xgb = mean_absolute_error(y_test_xgb, pred_xgb)
r2_xgb = r2_score(y_test_xgb, pred_xgb)

print("XGBoost RMSE:", rmse_xgb)
print("XGBoost MAE :", mae_xgb)
print("XGBoost R2  :", r2_xgb)

XGBoost RMSE: 14.009238783888176
XGBoost MAE : 10.98015790296999
XGBoost R2  : 0.9213061049327921


## Training Multi-Layer Perceptron (MLP) 

In [7]:
df_mlp = df_mlp.select_dtypes(include=["number"])

X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = ts_split(df_mlp)

# Scale tabular data
scaler_mlp = StandardScaler()
X_train_mlp_scaled = scaler_mlp.fit_transform(X_train_mlp)
X_test_mlp_scaled  = scaler_mlp.transform(X_test_mlp)

mlp_model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train_mlp_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dense(1)
])

d:\Projects\energy-forecasting\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
mlp_model.compile(optimizer="adam", loss="mse")
mlp_model.fit(X_train_mlp_scaled, y_train_mlp, epochs=25, batch_size=32, verbose=1)

pred_mlp = mlp_model.predict(X_test_mlp_scaled).flatten()

Epoch 1/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 19514.2344
Epoch 2/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 794.1052
Epoch 3/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 646.9674
Epoch 4/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 612.7626
Epoch 5/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 578.4688
Epoch 6/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 544.9061
Epoch 7/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 528.8756
Epoch 8/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 506.3365
Epoch 9/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 497.9416
Epoch 10/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 461.4678
Epoch 11/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 454.5944
Epoch 12/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 450.2085
Epoch 13/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 440.7553
Epoch 14/25
654/654 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 420.9569
Epoch 15/25
654/654 ━━━━━━━

In [9]:
# --- METRICS ---
rmse_mlp = np.sqrt(mean_squared_error(y_test_mlp, pred_mlp))
mae_mlp = mean_absolute_error(y_test_mlp, pred_mlp)
r2_mlp = r2_score(y_test_mlp, pred_mlp)

print("MLP RMSE:", rmse_mlp)
print("MLP MAE :", mae_mlp)
print("MLP R2  :", r2_mlp)

MLP RMSE: 14.249074915812807
MLP MAE : 11.074252135452086
MLP R2  : 0.9185885844915481


## Training LSTM (Deep Learning)

In [10]:
# LSTM Feature Columns
lstm_features = [
    "energy_consumption",
    "temperature_C",
    "humidity_pct",
    "hour_sin", "hour_cos",
    "dow_sin", "dow_cos"
]

In [11]:
SEQ_LEN = 24  # 24 hours sequence

# Convert to numpy
data = df_lstm[lstm_features + ["target"]].values

# Scale entire thing (LSTM friendly)
scaler_lstm = MinMaxScaler()
data_scaled = scaler_lstm.fit_transform(data)

In [12]:
# Create sequences
X_lstm, y_lstm = [], []
for i in range(len(data_scaled) - SEQ_LEN):
    X_lstm.append(data_scaled[i:i+SEQ_LEN, :-1])
    y_lstm.append(data_scaled[i+SEQ_LEN, -1])

X_lstm = np.array(X_lstm)
y_lstm = np.array(y_lstm)

# Split
split = int(len(X_lstm) * 0.8)
X_train_lstm, X_test_lstm = X_lstm[:split], X_lstm[split:]
y_train_lstm, y_test_lstm = y_lstm[:split], y_lstm[split:]

In [13]:
# LSTM Model
lstm_model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(SEQ_LEN, X_train_lstm.shape[2])),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(1)
])

lstm_model.compile(optimizer="adam", loss="mse")
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=25, batch_size=32, verbose=1)

pred_lstm_scaled = lstm_model.predict(X_test_lstm)

Epoch 1/25


d:\Projects\energy-forecasting\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


653/653 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0068
Epoch 2/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0023
Epoch 3/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0019
Epoch 4/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0017
Epoch 5/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0016
Epoch 6/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0016
Epoch 7/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0015
Epoch 8/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015
Epoch 9/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 10/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 11/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 12/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 13/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 14/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0014
Epoch 15/25
653/653 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0013


In [14]:
# Inverse scale target column only
dummy = np.zeros((len(pred_lstm_scaled), data_scaled.shape[1]))
dummy[:, -1] = pred_lstm_scaled.flatten()
pred_lstm = scaler_lstm.inverse_transform(dummy)[:, -1]

# True values
dummy2 = np.zeros((len(y_test_lstm), data_scaled.shape[1]))
dummy2[:, -1] = y_test_lstm.flatten()
true_lstm = scaler_lstm.inverse_transform(dummy2)[:, -1]

In [15]:
# --- METRICS ---
rmse_lstm = np.sqrt(mean_squared_error(true_lstm, pred_lstm))
mae_lstm = mean_absolute_error(true_lstm, pred_lstm)
r2_lstm = r2_score(true_lstm, pred_lstm)

print("LSTM RMSE:", rmse_lstm)
print("LSTM MAE :", mae_lstm)
print("LSTM R2  :", r2_lstm)

LSTM RMSE: 16.029500454050982
LSTM MAE : 12.563146805207172
LSTM R2  : 0.8968897594981229


In [16]:
print("\n====================================")
print("       📊 FINAL MODEL SCORES")
print("====================================")

print(f"LSTM     → RMSE: {rmse_lstm:.4f}, MAE: {mae_lstm:.4f}, R²: {r2_lstm:.4f}")
print(f"XGBoost  → RMSE: {rmse_xgb:.4f}, MAE: {mae_xgb:.4f}, R²: {r2_xgb:.4f}")
print(f"MLP      → RMSE: {rmse_mlp:.4f}, MAE: {mae_mlp:.4f}, R²: {r2_mlp:.4f}")


       📊 FINAL MODEL SCORES
LSTM     → RMSE: 16.0295, MAE: 12.5631, R²: 0.8969
XGBoost  → RMSE: 14.0092, MAE: 10.9802, R²: 0.9213
MLP      → RMSE: 14.2491, MAE: 11.0743, R²: 0.9186


In [17]:
model_scores = {
    "LSTM": rmse_lstm,
    "XGBoost": rmse_xgb,
    "MLP": rmse_mlp
}

best_model_name = min(model_scores, key=model_scores.get)
print("\nBEST MODEL BASED ON RMSE:", best_model_name)


BEST MODEL BASED ON RMSE: XGBoost


In [18]:
if best_model_name == "XGBoost":
    joblib.dump(model_xgb, "../models/best_model.pkl")
    joblib.dump(scaler_mlp, "../models/best_scaler.pkl")   # optional if using ML scaler
    print("✔ Saved best model: ../models/best_model.pkl (XGBoost)")

elif best_model_name == "MLP":
    mlp_model.save("../models/best_model.h5")
    joblib.dump(scaler_mlp, "../models/best_scaler.pkl")
    print("✔ Saved best model: ../models/best_model.h5 (MLP)")

elif best_model_name == "LSTM":
    lstm_model.save("../models/best_model.h5")
    joblib.dump(scaler_lstm, "../models/best_scaler.pkl")
    print("✔ Saved best model: ../models/best_model.h5 (LSTM)")

✔ Saved best model: ../models/best_model.pkl (XGBoost)
